In [1]:
#See correlations between different market indices since they open before US times
#^BUK100P, ^STOXX50E, ^N100, ^BFX, IMOEX.ME, ^N225, 000001.SS, ^STI, ^GSPTSE, ^KS11, ^BSESN, ^AXJO, ^HSI, ^N225

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import pandas_datareader.data as web
import tensorflow as tf
from openpyxl.workbook import Workbook as wb
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.utils.dataframe import dataframe_to_rows
import scipy
from scipy.stats.stats import pearsonr
import scipy.stats
import scipy.optimize
import scipy.spatial
import matplotlib.pyplot as plt
import seaborn as sb
from pylab import rcParams
from sklearn import linear_model
import statsmodels.api as sm
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

In [2]:
end = dt.datetime.now()
#change the number after the days
start = end - dt.timedelta(days=365*5)
symbols_list = 'IMOEX.ME', '^N100', '^N225', '^HSI', 'SPY', 'XLK', 'RSX', 'MSFT', 'ROSN.ME', '^RUT', 'GOOGL' 
symbols = []
#ticker = 'IMOEX.ME'
for ticker in symbols_list: 
    r = web.DataReader(ticker,'yahoo',start,end)
    r[f'{ticker} Adj Close'] = r['Adj Close']
    #r[f'{ticker} High'] = r['High']
    #r[f'{ticker} Low'] = r['Low']
    r[f'{ticker} Daily Return'] = (r['Adj Close'].pct_change())*100
    r[f'{ticker} Up or Down'] = int(False)
    r.loc[r[f'{ticker} Daily Return'] > 0, f'{ticker} Up or Down']=int(True)
    r1 = r[[f'{ticker} Adj Close']]
    #r1 = r[[f'{ticker} Up or Down']]
    #r1 = r[[f'{ticker} Adj Close',f'{ticker} Daily Return',f'{ticker} High',f'{ticker} Low',f'{ticker} Up or Down']]
    symbols.append(r1)
df = pd.concat(symbols, sort=False, axis=1)
df.dropna(how='any', inplace=True)
#df.to_excel('WorldIndicesData.xlsx')

In [3]:
#adjusted close correlations among major world indices
#The MOEX has the strongest relationship with SPY
df.corr()

,IMOEX.ME Adj Close,^N100 Adj Close,^N225 Adj Close,^HSI Adj Close,SPY Adj Close,XLK Adj Close,RSX Adj Close,MSFT Adj Close,ROSN.ME Adj Close,^RUT Adj Close,GOOGL Adj Close
IMOEX.ME Adj Close,1.000000,0.662106,0.634624,0.499258,0.936154,0.940664,0.911538,0.942026,0.927925,0.713394,0.902839
^N100 Adj Close,0.662106,1.000000,0.901886,0.809744,0.811192,0.733344,0.826036,0.624304,0.610417,0.890364,0.738954
^N225 Adj Close,0.634624,0.901886,1.000000,0.842775,0.805419,0.740131,0.758739,0.644915,0.597806,0.886465,0.749040
^HSI Adj Close,0.499258,0.809744,0.842775,1.000000,0.707847,0.630438,0.717196,0.506542,0.442087,0.846469,0.647147
SPY Adj Close,0.936154,0.811192,0.805419,0.707847,1.000000,0.982683,0.931181,0.939432,0.876724,0.864009,0.968241
XLK Adj Close,0.940664,0.733344,0.740131,0.630438,0.982683,1.000000,0.889317,0.981184,0.853097,0.775235,0.968210
RSX Adj Close,0.911538,0.826036,0.758739,0.717196,0.931181,0.889317,1.000000,0.827624,0.830719,0.847777,0.863169
MSFT Adj Close,0.942026,0.624304,0.644915,0.506542,0.939432,0.981184,0.827624,1.000000,0.840715,0.656890,0.927706
ROSN.ME Adj Close,0.927925,0.610417,0.597806,0.442087,0.876724,0.853097,0.830719,0.840715,1.000000,0.746664,0.851512
^RUT Adj Close,0.713394,0.890364,0.886465,0.846469,0.864009,0.775235,0.847777,0.656890,0.746664,1.000000,0.814613


In [5]:
### Run a linear regression model between the MOEX and SPY ### As well as ^N100 and ^N225
### Russia's market is open before ours so seeing where they are could help 
### judge where certain securities like SPY will be when our market opens

In [4]:
#finding multiple regression to SPY
#Russian markets open up before and if RSX and MOEX are up, good chance SPY will be up, ^N100 or ^N225
y = df[['XLK Adj Close']].values
x = df[['IMOEX.ME Adj Close', '^N100 Adj Close']] 
regr = linear_model.LinearRegression()
regr.fit(x, y)
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)
model = sm.OLS(y, x).fit()
predictions = model.predict(x) 
print_model = model.summary()
print(print_model)

Intercept: 
 [-54.70241991]
Coefficients: 
 [[0.03510542 0.0372825 ]]
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.987
Model:                            OLS   Adj. R-squared (uncentered):              0.987
Method:                 Least Squares   F-statistic:                          4.287e+04
Date:                Mon, 11 May 2020   Prob (F-statistic):                        0.00
Time:                        14:34:34   Log-Likelihood:                         -3664.9
No. Observations:                1093   AIC:                                      7334.
Df Residuals:                    1091   BIC:                                      7344.
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                         coef    std err          

In [5]:
#finding multiple regression to SPY
#Russian markets open up before and if ^N225 and MOEX are up, good chance SPY will be up, ^N100 or ^N225
#Tokyo is 13 hours ahead and Moscow is 7 hours ahead
y = df[['SPY Adj Close']].values
x = df[['^N225 Adj Close','IMOEX.ME Adj Close']] 
regr = linear_model.LinearRegression()
regr.fit(x, y)
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)
model = sm.OLS(y, x).fit()
predictions = model.predict(x) 
print_model = model.summary()
print(print_model)

Intercept: 
 [-65.53869782]
Coefficients: 
 [[0.00663834 0.07770881]]
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.998
Model:                            OLS   Adj. R-squared (uncentered):              0.998
Method:                 Least Squares   F-statistic:                          2.387e+05
Date:                Mon, 11 May 2020   Prob (F-statistic):                        0.00
Time:                        14:34:36   Log-Likelihood:                         -4234.2
No. Observations:                1093   AIC:                                      8472.
Df Residuals:                    1091   BIC:                                      8482.
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                         coef    std err          

In [6]:
y = df[['^RUT Adj Close']].values
x = df[['IMOEX.ME Adj Close','^N225 Adj Close']] 
regr = linear_model.LinearRegression()
regr.fit(x, y)
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)
model = sm.OLS(y, x).fit()
predictions = model.predict(x) 
print_model = model.summary()
print(print_model)

Intercept: 
 [-146.30376758]
Coefficients: 
 [[0.12681606 0.06266071]]
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.997
Model:                            OLS   Adj. R-squared (uncentered):              0.997
Method:                 Least Squares   F-statistic:                          1.623e+05
Date:                Mon, 11 May 2020   Prob (F-statistic):                        0.00
Time:                        14:34:39   Log-Likelihood:                         -6366.1
No. Observations:                1093   AIC:                                  1.274e+04
Df Residuals:                    1091   BIC:                                  1.275e+04
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                         coef    std err         

In [11]:
def worldIndice():
    '''This function is used to make an excel I can use with 
    R programming language that has an easier stats package
    that I can compare the data'''
    end = dt.datetime.now()
    #change the number after the days
    start = end - dt.timedelta(days=365*5)
    symbols_list = [
        'IMOEX.ME','QQQ','SQQQ','^N100','^N225','^AXJO','ULVR.L','PRU.L','LSE.L','^VIX','^HSI','SPY','XLK','RSX',
        'MSFT','ROSN.ME','^RUT','GOOGL','FB','AMZN','SNAP','SBRCY','000001.SS','^STOXX','^STOXX50E','AIR.PA',
        '6758.T','GC=F','INTC','TCEHY','0700.HK','HSBA.L','VOD.L', '7203.T','TM', 'HSBC'
    ]
    symbols = []
    #ticker = 'IMOEX.ME'
    for ticker in symbols_list: 
        r = web.DataReader(ticker,'yahoo',start,end)
        r[f'{ticker} Adj Close'] = r['Adj Close']
        r[f'{ticker} Daily Return'] = (r['Adj Close'].pct_change())*100
        r[f'{ticker} Up or Down'] = int(False)
        r.loc[r[f'{ticker} Daily Return'] > 0, f'{ticker} Up or Down']=int(True)
        r1 = r[[f'{ticker} Adj Close',f'{ticker} Up or Down',f'{ticker} Daily Return']]
        symbols.append(r1)
    df = pd.concat(symbols, sort=False, axis=1)
    df.dropna(how='any', inplace=True)
    #will move this to a working directory to be used in R because I like of some of the stattistical packages there 
    df.to_excel('WorldIndices.xlsx')

In [12]:
worldIndice()

In [13]:
#https://github.com/timestocome/Test-stock-prediction-algorithms/blob/master/StockMarketLinearRegression/LogisticRegression_Classifier.py
#take correlations found from R code and apply to machine learning model
data = pd.read_excel('WorldIndices.xlsx')
data.set_index('Date', inplace=True)
dfBin = data.iloc[:,1:len(data):3]
#print(dfBin)
weeklyDf = data.resample('W-MON').sum()
#drop adjusted close prices because might make model innaccurate
drop = weeklyDf.iloc[:,0:len(data):3]
weeklyBin = dfBin.resample('W-MON').sum()
weeklyDf = weeklyDf.drop(drop,axis=1)
#weeklyDf.columns
dfBin.columns
#get binary data into weekly format

Index(['IMOEX.ME Up or Down', 'QQQ Up or Down', 'SQQQ Up or Down',
       '^N100 Up or Down', '^N225 Up or Down', '^AXJO Up or Down',
       'ULVR.L Up or Down', 'PRU.L Up or Down', 'LSE.L Up or Down',
       '^VIX Up or Down', '^HSI Up or Down', 'SPY Up or Down',
       'XLK Up or Down', 'RSX Up or Down', 'MSFT Up or Down',
       'ROSN.ME Up or Down', '^RUT Up or Down', 'GOOGL Up or Down',
       'FB Up or Down', 'AMZN Up or Down', 'SNAP Up or Down',
       'SBRCY Up or Down', '000001.SS Up or Down', '^STOXX Up or Down',
       '^STOXX50E Up or Down', 'AIR.PA Up or Down', '6758.T Up or Down',
       'GC=F Up or Down', 'INTC Up or Down', 'TCEHY Up or Down',
       '0700.HK Up or Down', 'HSBA.L Up or Down', 'VOD.L Up or Down',
       '7203.T Up or Down', 'TM Up or Down', 'HSBC Up or Down'],
      dtype='object')

In [14]:
###Weekly Matrix showing correlations between QQQ ETF and rest of indices and stocks###
print("CORRELATION MATRIX")
correlationMatrix = weeklyDf.corr()
correlationMatrix = correlationMatrix[correlationMatrix>.4]
sig = correlationMatrix['QQQ Daily Return'].dropna()
sig.sort_values(ascending=False)

CORRELATION MATRIX


QQQ Daily Return          1.000000
XLK Daily Return          0.978669
MSFT Daily Return         0.935599
SPY Daily Return          0.926654
GOOGL Daily Return        0.870148
^RUT Daily Return         0.870009
^N100 Daily Return        0.818406
INTC Daily Return         0.817670
^STOXX Daily Return       0.808573
^STOXX50E Daily Return    0.804636
FB Daily Return           0.786211
AMZN Daily Return         0.781387
PRU.L Daily Return        0.728105
TM Daily Return           0.722681
^N225 Daily Return        0.678450
LSE.L Daily Return        0.663107
TCEHY Daily Return        0.654634
RSX Daily Return          0.611947
HSBC Daily Return         0.576129
IMOEX.ME Daily Return     0.568590
VOD.L Daily Return        0.530270
7203.T Daily Return       0.498697
^HSI Daily Return         0.487707
ULVR.L Daily Return       0.476896
0700.HK Daily Return      0.474457
ROSN.ME Daily Return      0.473597
SPY Up or Down            0.472638
QQQ Up or Down            0.466629
AIR.PA Daily Return 

In [15]:
###Weekly Matrix showing covariations between QQQ ETF and rest of indices and stocks###
print("COVARIANCE MATRIX")
covarianceMatrix = weeklyDf.cov()
covarianceMatrix = covarianceMatrix[covarianceMatrix > 1]
sigCov = (covarianceMatrix['QQQ Daily Return'].dropna())
sigCov.sort_values(ascending = False)

COVARIANCE MATRIX


PRU.L Daily Return        9.038312
FB Daily Return           8.905136
INTC Daily Return         8.878924
SNAP Daily Return         8.348055
MSFT Daily Return         8.345131
XLK Daily Return          8.145789
^RUT Daily Return         8.007569
GOOGL Daily Return        7.742893
QQQ Daily Return          7.606938
TCEHY Daily Return        7.325602
SPY Daily Return          6.810597
AMZN Daily Return         6.717057
AIR.PA Daily Return       6.364940
LSE.L Daily Return        6.211397
^N100 Daily Return        5.437931
VOD.L Daily Return        5.397309
RSX Daily Return          5.395944
^STOXX50E Daily Return    5.376100
ROSN.ME Daily Return      5.324567
^STOXX Daily Return       5.086020
TM Daily Return           4.610769
SBRCY Daily Return        4.413243
0700.HK Daily Return      4.403556
HSBC Daily Return         4.005097
^N225 Daily Return        3.985768
IMOEX.ME Daily Return     3.954418
6758.T Daily Return       3.851611
7203.T Daily Return       3.198198
ULVR.L Daily Return 

In [17]:
###Weekly Up and Down Multiple Regression###
y = weeklyDf[['SPY Up or Down']].values
x = weeklyDf[['IMOEX.ME Up or Down','^N100 Up or Down','^STOXX Up or Down']]  
regr = linear_model.LinearRegression()
regr.fit(x, y)
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)
model = sm.OLS(y, x).fit()
predictions = model.predict(x) 
print_model = model.summary()
print(print_model)

Intercept: 
 [0.60920723]
Coefficients: 
 [[0.22128251 0.36529714 0.11591637]]
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.830
Model:                            OLS   Adj. R-squared (uncentered):              0.827
Method:                 Least Squares   F-statistic:                              267.5
Date:                Mon, 11 May 2020   Prob (F-statistic):                    6.39e-63
Time:                        14:39:53   Log-Likelihood:                         -220.19
No. Observations:                 167   AIC:                                      446.4
Df Residuals:                     164   BIC:                                      455.7
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                          coef    std err

In [18]:
####Daily Correlations####

In [54]:
###Daily Up and Down###
data = pd.read_excel('WorldIndices.xlsx')
data.set_index('Date', inplace=True)
dfBin = data.iloc[:,1:len(data):3]
#drop adjusted close prices because might make model innaccurate
drop = data.iloc[:,0:len(data):3]
df = data.drop(drop,axis=1)

In [55]:
###Daily Matrix showing correlations between SPY ETF and rest of indices and stocks###
print("CORRELATION MATRIX")
correlationMatrix = df.corr()
correlationMatrix = correlationMatrix[correlationMatrix>.3]
sig = correlationMatrix['TM Up or Down'].dropna()
sig.sort_values(ascending=False)

CORRELATION MATRIX


TM Up or Down          1.000000
TM Daily Return        0.665466
7203.T Up or Down      0.484776
7203.T Daily Return    0.467321
QQQ Daily Return       0.351761
XLK Daily Return       0.349178
SPY Daily Return       0.347651
SPY Up or Down         0.336494
^RUT Daily Return      0.324234
XLK Up or Down         0.307989
GOOGL Daily Return     0.306400
TCEHY Daily Return     0.306222
Name: TM Up or Down, dtype: float64

In [56]:
###Daily Matrix showing covariations between SPY ETF and rest of indices and stocks###
print("COVARIANCE MATRIX")
covarianceMatrix = df.cov()
covarianceMatrix = covarianceMatrix[covarianceMatrix > .1]
sigCov = (covarianceMatrix['SPY Up or Down'].dropna())
sigCov.sort_values(ascending = False)

COVARIANCE MATRIX


TCEHY Daily Return        0.478997
MSFT Daily Return         0.441637
INTC Daily Return         0.429911
XLK Daily Return          0.425314
FB Daily Return           0.424443
AMZN Daily Return         0.416957
GOOGL Daily Return        0.411490
QQQ Daily Return          0.403543
SBRCY Daily Return        0.395678
SNAP Daily Return         0.385637
^RUT Daily Return         0.381899
RSX Daily Return          0.378209
AIR.PA Daily Return       0.351275
SPY Daily Return          0.340091
PRU.L Daily Return        0.276433
HSBC Daily Return         0.250432
SPY Up or Down            0.246828
TM Daily Return           0.232148
^STOXX50E Daily Return    0.212946
^N100 Daily Return        0.212777
^STOXX Daily Return       0.192339
LSE.L Daily Return        0.180773
QQQ Up or Down            0.171641
XLK Up or Down            0.165412
HSBA.L Daily Return       0.154876
0700.HK Daily Return      0.154257
VOD.L Daily Return        0.148389
^RUT Up or Down           0.140139
6758.T Daily Return 

In [57]:
###Daily World Index Daily Pos / Neg Return Multiple Regression###
y = df[['SPY Up or Down']].values
#x = df[['IMOEX.ME Up or Down','^N225 Up or Down','^STOXX50E Up or Down']]  
x = df[['IMOEX.ME Up or Down','^N100 Up or Down','^STOXX50E Up or Down']]  
regr = linear_model.LinearRegression()
regr.fit(x, y)
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)
model = sm.OLS(y, x).fit()
predictions = model.predict(x) 
print_model = model.summary()
print(print_model)

Intercept: 
 [0.26609209]
Coefficients: 
 [[0.14195724 0.2929779  0.1092886 ]]
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.604
Model:                            OLS   Adj. R-squared (uncentered):              0.602
Method:                 Least Squares   F-statistic:                              285.9
Date:                Mon, 11 May 2020   Prob (F-statistic):                   1.03e-112
Time:                        15:09:04   Log-Likelihood:                         -377.13
No. Observations:                 566   AIC:                                      760.3
Df Residuals:                     563   BIC:                                      773.3
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                           coef    std er

In [58]:
### Conditional Probability Between SPY and ^N100 is 73%
'''WHEN THE ^N100 IS UP, 73% chance that SPY will be up (.67+.06) (.06 is added in because ^STOXX can be up or
 down for the day and not matter)'''
df.groupby('SPY Up or Down').count()['XLK Up or Down'] / len(df)
#df.groupby('XLK Up or Down').count()['SPY Up or Down'] / len(df)
(df.groupby(['SPY Up or Down','^N100 Up or Down']).count() / df.groupby('SPY Up or Down').count())['XLK Up or Down']
#(df.groupby(['XLK Up or Down', '^STOXX50E Up or Down']).count() / df.groupby('XLK Up or Down').count())['SPY Up or Down']

SPY Up or Down  ^N100 Up or Down
0               0                   0.682731
                1                   0.317269
1               0                   0.268139
                1                   0.731861
Name: XLK Up or Down, dtype: float64

In [59]:
### Conditional Probability Between XLK and ^N100 is 70.7452% ###

'''WHEN THE ^N100 IS UP, ~71% chance that SPY will be up (.67+.06) (.06 is added in because ^STOXX can be up or
 down for the day and not matter)'''
df.groupby('^N100 Up or Down').count()['SPY Up or Down'] / len(df)
#df.groupby('XLK Up or Down').count()['SPY Up or Down'] / len(df)
(df.groupby(['^N100 Up or Down','XLK Up or Down']).count() / df.groupby('^N100 Up or Down').count())['SPY Up or Down']
#(df.groupby(['XLK Up or Down', '^STOXX50E Up or Down']).count() / df.groupby('XLK Up or Down').count())['SPY Up or Down']
#IMOEX.ME Up or Down','^N100 Up or Down','^STOXX Up or Down w/ SPY

^N100 Up or Down  XLK Up or Down
0                 0                 0.619608
                  1                 0.380392
1                 0                 0.292605
                  1                 0.707395
Name: SPY Up or Down, dtype: float64

In [78]:
### Conditional Probability Between Toyota's NYSE and Japan Equity 76.5957 ###

df.groupby('7203.T Up or Down').count()['SPY Up or Down'] / len(df)
(df.groupby(['7203.T Up or Down','TM Up or Down']).count() / df.groupby('7203.T Up or Down').count())['SPY Up or Down']

7203.T Up or Down  TM Up or Down
0                  0                0.718310
                   1                0.281690
1                  0                0.234043
                   1                0.765957
Name: SPY Up or Down, dtype: float64

In [61]:
###QQQ conditional probability
df.groupby('^N100 Up or Down').count()['XLK Up or Down'] / len(df)
(df.groupby(['^N100 Up or Down','^STOXX Up or Down','QQQ Up or Down']).count() / df.groupby('^N100 Up or Down').count())['XLK Up or Down']

^N100 Up or Down  ^STOXX Up or Down  QQQ Up or Down
0                 0                  0                 0.560784
                                     1                 0.345098
                  1                  0                 0.047059
                                     1                 0.047059
1                 0                  0                 0.022508
                                     1                 0.057878
                  1                  0                 0.254019
                                     1                 0.665595
Name: XLK Up or Down, dtype: float64

In [62]:
#Binary Conditional Probability with QQQ and ^N100 (whether the stock was up or down for the day) 
df.groupby('^N100 Up or Down').count()['XLK Up or Down'] / len(df)
(df.groupby(['^N100 Up or Down','QQQ Up or Down']).count() / df.groupby('^N100 Up or Down').count())['XLK Up or Down']

^N100 Up or Down  QQQ Up or Down
0                 0                 0.607843
                  1                 0.392157
1                 0                 0.276527
                  1                 0.723473
Name: XLK Up or Down, dtype: float64

In [63]:
#Binary Conditional Probability with QQQ and ^STOXX (whether the stock was up or down for the day) 
df.groupby('^STOXX Up or Down').count()['XLK Up or Down'] / len(df)
(df.groupby(['^STOXX Up or Down','QQQ Up or Down']).count() / df.groupby('^STOXX Up or Down').count())['XLK Up or Down']

^STOXX Up or Down  QQQ Up or Down
0                  0                 0.585938
                   1                 0.414062
1                  0                 0.293548
                   1                 0.706452
Name: XLK Up or Down, dtype: float64

In [83]:
df.groupby('IMOEX.ME Up or Down').count()['XLK Up or Down'] / len(df)
(df.groupby(['IMOEX.ME Up or Down','QQQ Up or Down']).count() / df.groupby('IMOEX.ME Up or Down').count())['XLK Up or Down']

IMOEX.ME Up or Down  QQQ Up or Down
0                    0                 0.524164
                     1                 0.475836
1                    0                 0.336700
                     1                 0.663300
Name: XLK Up or Down, dtype: float64

In [93]:
#New dataframe to predict if two conditionals = a higher probability
df1 = df[['^STOXX Up or Down','XLK Up or Down','QQQ Up or Down','^N100 Up or Down','SPY Up or Down','IMOEX.ME Up or Down']]
df1['^STOXX and ^N100'] = df1['^STOXX Up or Down'] & df1['^N100 Up or Down']
df1['IMOEX.ME and ^N100'] = df1['IMOEX.ME Up or Down'] & df1['^N100 Up or Down']
df1['IMOEX.ME and ^N100 and ^STOXX'] = df1['IMOEX.ME Up or Down'] & df1['^N100 Up or Down'] & df1['^STOXX Up or Down']
df1

,^STOXX Up or Down,XLK Up or Down,QQQ Up or Down,^N100 Up or Down,SPY Up or Down,IMOEX.ME Up or Down,^STOXX and ^N100,IMOEX.ME and ^N100,IMOEX.ME and ^N100 and ^STOXX
Date,,,,,,,,,
2017-03-06,0,0,0,0,0,0,0,0,0
2017-03-07,0,0,0,0,0,0,0,0,0
2017-03-09,1,0,1,1,1,0,1,0,0
2017-03-13,1,1,1,1,1,1,1,1,1
2017-03-14,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
2020-04-23,1,0,0,1,0,1,1,1,1
2020-04-24,0,1,1,0,1,0,0,0,0
2020-04-27,1,1,1,1,1,1,1,1,1


In [75]:
#barely (other SPY binary to N100 binary is better because it happens more often then having both to be Up)
df1.groupby('^STOXX and ^N100').count()['XLK Up or Down'] / len(df1)
(df1.groupby(['^STOXX and ^N100','SPY Up or Down']).count() / df1.groupby('^STOXX and ^N100').count())['XLK Up or Down']

^STOXX and ^N100  SPY Up or Down
0                 0                 0.628571
                  1                 0.371429
1                 0                 0.255245
                  1                 0.744755
Name: XLK Up or Down, dtype: float64

In [82]:
###could do both SQQQ and QQQ with hedging (SQQQ is just the short version of being long QQQ)
df1.groupby('^STOXX and ^N100').count()['XLK Up or Down'] / len(df1)
(df1.groupby(['^STOXX and ^N100','QQQ Up or Down']).count() / df1.groupby('^STOXX and ^N100').count())['XLK Up or Down']

^STOXX and ^N100  QQQ Up or Down
0                 0                 0.578571
                  1                 0.421429
1                 0                 0.276224
                  1                 0.723776
Name: XLK Up or Down, dtype: float64

In [89]:
df1.groupby('IMOEX.ME and ^N100').count()['XLK Up or Down'] / len(df1)
(df1.groupby(['IMOEX.ME and ^N100','QQQ Up or Down']).count() / df1.groupby('IMOEX.ME and ^N100').count())['XLK Up or Down']

IMOEX.ME and ^N100  QQQ Up or Down
0                   0                 0.517615
                    1                 0.482385
1                   0                 0.253807
                    1                 0.746193
Name: XLK Up or Down, dtype: float64

In [90]:
df1.groupby('IMOEX.ME and ^N100').count()['XLK Up or Down'] / len(df1)
(df1.groupby(['IMOEX.ME and ^N100','SPY Up or Down']).count() / df1.groupby('IMOEX.ME and ^N100').count())['XLK Up or Down']

IMOEX.ME and ^N100  SPY Up or Down
0                   0                 0.563686
                    1                 0.436314
1                   0                 0.208122
                    1                 0.791878
Name: XLK Up or Down, dtype: float64

In [92]:
df1.groupby('IMOEX.ME and ^N100').count()['SPY Up or Down'] / len(df1)
(df1.groupby(['IMOEX.ME and ^N100','XLK Up or Down']).count() / df1.groupby('IMOEX.ME and ^N100').count())['SPY Up or Down']

IMOEX.ME and ^N100  XLK Up or Down
0                   0                 0.533875
                    1                 0.466125
1                   0                 0.263959
                    1                 0.736041
Name: SPY Up or Down, dtype: float64

In [96]:
df1.groupby('IMOEX.ME and ^N100 and ^STOXX').count()['XLK Up or Down'] / len(df1)
(df1.groupby(['IMOEX.ME and ^N100 and ^STOXX','SPY Up or Down']).count() / df1.groupby('IMOEX.ME and ^N100 and ^STOXX').count())['XLK Up or Down']

IMOEX.ME and ^N100 and ^STOXX  SPY Up or Down
0                              0                 0.554688
                               1                 0.445312
1                              0                 0.197802
                               1                 0.802198
Name: XLK Up or Down, dtype: float64

In [97]:
df1.groupby('IMOEX.ME and ^N100 and ^STOXX').count()['XLK Up or Down'] / len(df1)
(df1.groupby(['IMOEX.ME and ^N100 and ^STOXX','QQQ Up or Down']).count() / df1.groupby('IMOEX.ME and ^N100 and ^STOXX').count())['XLK Up or Down']

IMOEX.ME and ^N100 and ^STOXX  QQQ Up or Down
0                              0                 0.513021
                               1                 0.486979
1                              0                 0.241758
                               1                 0.758242
Name: XLK Up or Down, dtype: float64

In [98]:
df1.groupby('IMOEX.ME and ^N100 and ^STOXX').count()['SPY Up or Down'] / len(df1)
(df1.groupby(['IMOEX.ME and ^N100 and ^STOXX','XLK Up or Down']).count() / df1.groupby('IMOEX.ME and ^N100 and ^STOXX').count())['SPY Up or Down']

IMOEX.ME and ^N100 and ^STOXX  XLK Up or Down
0                              0                 0.528646
                               1                 0.471354
1                              0                 0.252747
                               1                 0.747253
Name: SPY Up or Down, dtype: float64

In [ ]:
#######Conclusions#######
### SPY: SPY would be up 80% of the time if ^N100,^STOXX, and IMOEX.ME is up
### SPY: SPY would be up 79% of the time if ^N100 and IMOEX.ME is up
### SPY: SPY would be up 73% of the time if ^N100 is up
### SPY: SPY would be up 74% of the time if ^N100 and ^STOXX are up
### QQQ: QQQ would be up 76% of the time when ^N100,^STOXX, and IMOEX.ME are up
### QQQ: QQQ is up up 72% of the time when ^N100 is up
### QQQ: QQQ would be up 74% of the time when ^N100 and ^STOXX are up
### XLK: XLK is up 75% of the time when ^N100,IMOEX.ME and ^STOXX are up
### XLK: XLK is up 70.74% of the time when ^N100 is up
### XLK: XLK is up 71% of the time when ^N100 and ^STOXX are up
### T-Mobile: Is up 76.59% of the time when their equity in Tokyo was up

In [ ]:
###Further Studies would be to include the RSI and Stochastic Oscillator
###When the buy signal is there and a world indice is up prior to NYSE or NASDAQ starting the day